In [1]:
import pandas as pd
import datetime

In [2]:
# OrderBookPlots2002_df = pd.read_csv('./resources/2002.csv')
# OrderBookPlots2002_df = OrderBookPlots2002_df[OrderBookPlots2002_df.DATE.isin(['2002-08-12', '2002-08-13'])]
# OrderBookPlots2002_df

In [3]:
# OrderBookPlots2002_df['TRADENO'] = OrderBookPlots2002_df['TRADENO'].astype(pd.Int64Dtype())

In [4]:
# OrderBookPlots2002_df.sort_values(by=['NO'], inplace=True)

In [5]:
# OrderBookPlots2002_df[(OrderBookPlots2002_df.PRICE != OrderBookPlots2002_df.TRADEPRICE) &
#                  pd.notnull(OrderBookPlots2002_df.TRADEPRICE)]

In [6]:
OrderBookPlots2007_df = pd.read_csv('./resources/LKOH_SBER_GMKN_2007_10.csv')
OrderBookPlots2007_df = OrderBookPlots2007_df[(OrderBookPlots2007_df.SECCODE == 'LKOH') & 
                                              (OrderBookPlots2007_df.DATE == '2007-10-08')]
# OrderBookPlots2007_df = OrderBookPlots2007_df[OrderBookPlots2007_df.DATE.isin(['2007-10-01', '2007-10-02'])]
# OrderBookPlots2007_df

In [7]:
OrderBookPlots2007_df['TRADENO'] = OrderBookPlots2007_df['TRADENO'].astype(pd.Int64Dtype())

In [8]:
OrderBookPlots2007_df.sort_values(by=['NO'], inplace=True)

In [9]:
OrderBookPlots2007_df[(OrderBookPlots2007_df.PRICE != OrderBookPlots2007_df.TRADEPRICE) &
                 pd.notnull(OrderBookPlots2007_df.TRADEPRICE)]

,Unnamed: 0,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,DATE,OBPLOTNO
1544302,17145,17146,LKOH,S,103001000,17122,2,2179.99,40,332168465,2180.00,2007-10-08,1399
1544318,17169,17170,LKOH,S,103001000,17071,2,2178.90,10,332168477,2180.00,2007-10-08,1399
1544321,17192,17193,LKOH,S,103001000,17081,2,2179.99,5,332168488,2180.00,2007-10-08,1399
1544324,17213,17214,LKOH,B,103001000,17130,2,0.00,50,332168499,2181.99,2007-10-08,129
1544326,17215,17216,LKOH,B,103001000,17053,2,2200.00,3,332168500,2181.99,2007-10-08,129
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1834830,910304,910305,LKOH,S,174458000,395515,2,0.00,1,332355810,2175.60,2007-10-08,79324
1834832,910306,910307,LKOH,S,174458000,395515,2,0.00,84,332355811,2175.50,2007-10-08,79324
1834835,910309,910310,LKOH,S,174458000,395496,2,0.00,10,332355812,2175.50,2007-10-08,79324
1834836,910326,910327,LKOH,B,174458000,395498,2,0.00,1,332355821,2176.02,2007-10-08,79201


In [10]:
# OrderBookPlots2002_2000ms_df = OrderBookPlots2002_df[(OrderBookPlots2002_df.TIME >= 103002000) & (OrderBookPlots2002_df.TIME <= 103003000)]
# OrderBookPlots2002_2000ms_df

In [11]:
# OrderBookPlots2002_2000ms_df = OrderBookPlots2002_df[(OrderBookPlots2002_df.TIME >= 103002000) & (OrderBookPlots2002_df.TIME <= 103003000)]
def form_datetimemlls(df):
    scnd_size_df = pd.DataFrame(df.groupby('TIME').size())
    scnd_size_df.reset_index(inplace=True)
    scnd_size_df = scnd_size_df.rename(columns = {0:'SCNDSIZE'})
    df = df.merge(scnd_size_df, on='TIME')
    df['SEC_IND'] = df.groupby(['TIME']).cumcount()
    df['MILLISS'] = round(df['SEC_IND'] / df['SCNDSIZE'] * 1000)
    df['MILLISS'] = df['MILLISS'].astype(int)
    df.drop(['SCNDSIZE', 'SEC_IND'], axis=1, inplace=True)
    df['DATETIME'] = df.apply(lambda row: datetime.datetime.strptime(row.DATE, '%Y-%m-%d'), axis=1)
    df['DATETIMEMLLS'] = df.apply(lambda row: datetime.datetime(row.DATETIME.year,
                                                               row.DATETIME.month,
                                                               row.DATETIME.day,
                                                               int(row.TIME // 1e7),
                                                               int((row.TIME // 1e5) % 100),
                                                               int((row.TIME // 1e3) % 100)),
                                  axis=1)
    df['DATETIMEMLLS'] = df.apply(lambda row: row.DATETIMEMLLS +
                                 datetime.timedelta(milliseconds=row.MILLISS),
                                 axis=1)
    df.drop(['DATETIME', 'MILLISS', 'TIME'], axis=1, inplace=True)
    return(df)
# OrderBookPlots2002_2000ms_df = form_datetimemlls(OrderBookPlots2002_2000ms_df)
# OrderBookPlots2002_2000ms_df

In [12]:
# OrderBookPlots2002_df = form_datetimemlls(OrderBookPlots2002_df)

In [13]:
OrderBookPlots2007_df = form_datetimemlls(OrderBookPlots2007_df)

In [14]:
class OBPlots2OBPlots_Attributes:
    def __init__(self, OBPs_df):
        self.OBPs_df = OBPs_df
        self.secs = pd.unique(self.OBPs_df.SECCODE)
        self.dates = pd.unique(self.OBPs_df.DATE)
#         print(self.dates)
        self.obplotnos = pd.unique(self.OBPs_df.OBPLOTNO)
        self.obp_atts_dic = {
            sec:datedic for (sec, datedic) in zip(
                self.secs,
                [{
                    date:obps_attsdic for (date, obps_attsdic) in zip(
                        self.dates, 
                        [{} for i in range(len(self.dates))]
                    )
                } for i in range(len(self.secs))]
            )
        }
        self.iterate_ob_rows()
    
    # Accordin to specs ACTION 0 shows balance of order, not cancelled volune of the order, IT'S WRONG
    # Actually, it shows cancelled volune of the order
    def ob2curatt(self, SECCODE, DATE, OBPLOTNO, BUYSELL, ACTION, PRICE, VOLUME, TRADEPRICE):
        if (BUYSELL == 'B') & (ACTION == 0):
            CBOVOL = self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV']['CBOVOL'] + (PRICE * VOLUME)
            self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV']['CBOVOL'] = CBOVOL
            return({'ATT': 'CBOVOL', 'VAL': CBOVOL})
        elif (BUYSELL == 'S') & (ACTION == 0):
            CSOVOL = self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV']['CSOVOL'] + (PRICE * VOLUME)
            self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV']['CSOVOL'] = CSOVOL
            return({'ATT': 'CSOVOL', 'VAL': CSOVOL})
        elif (BUYSELL == 'B') & (ACTION == 1):
            BOVOL = self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV']['BOVOL'] + (PRICE * VOLUME)
            self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV']['BOVOL'] = BOVOL
            return({'ATT': 'BOVOL', 'VAL': BOVOL})
        elif (BUYSELL == 'S') & (ACTION == 1):
            SOVOL = self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV']['SOVOL'] + (PRICE * VOLUME)
            self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV']['SOVOL'] = SOVOL
            return({'ATT': 'SOVOL', 'VAL': SOVOL})
        elif (BUYSELL == 'B') & (ACTION == 2):
            BTVOL = self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV']['BTVOL'] + (TRADEPRICE * VOLUME)
            self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV']['BTVOL'] = BTVOL
            return({'ATT': 'BTVOL', 'VAL': BTVOL})
        elif (BUYSELL == 'S') & (ACTION == 2):
            STVOL = self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV']['STVOL'] + (TRADEPRICE * VOLUME)
            self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV']['STVOL'] = STVOL
            return({'ATT': 'STVOL', 'VAL': STVOL})
        
    def ob_row2obp_atts(self, SECCODE, DATE, NO, BUYSELL, ORDERNO,
                        ACTION, PRICE, VOLUME, TRADEPRICE, OBPLOTNO,
                        DATETIMEMLLS):
        if OBPLOTNO not in self.obp_atts_dic[SECCODE][DATE]:
            self.obp_atts_dic[SECCODE][DATE][OBPLOTNO] = {}
            self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV'] = {
                'CBOVOL': 0, # cancel buy order volume
                'CSOVOL': 0, # cancel sell order volume
                'BOVOL': 0, # buy order volume
                'SOVOL': 0, # sell order volume
                'BTVOL': 0, # buy trade volumne
                'STVOL': 0, # sell trade volume
                'TBNET': 0, # trade volumne - buy volume <= 0
                'TSNET': 0, # trade volume - sell volume >= 0
                'OBP_BAL': 0 # oreder-book-plot balance
            }
            
        self.obp_atts_dic[SECCODE][DATE][OBPLOTNO][NO] = self.ob2curatt(SECCODE, DATE, OBPLOTNO, BUYSELL, ACTION, 
                                                                    PRICE, VOLUME, TRADEPRICE)
        self.obp_atts_dic[SECCODE][DATE][OBPLOTNO][NO]['DATETIMEMLLS'] = DATETIMEMLLS
#         print(SECCODE)
#         print(self.obp_atts[SECCODE])
        
    def iterate_ob_rows(self):
        [self.ob_row2obp_atts(SECCODE, DATE, NO, BUYSELL, ORDERNO, 
                              ACTION, PRICE, VOLUME, TRADEPRICE, OBPLOTNO,
                              DATETIMEMLLS)
        for (SECCODE, DATE, NO, BUYSELL, ORDERNO,
             ACTION, PRICE, VOLUME, TRADEPRICE, OBPLOTNO,
             DATETIMEMLLS)
         in zip(self.OBPs_df.SECCODE,
                self.OBPs_df.DATE,
                self.OBPs_df.NO,
                self.OBPs_df.BUYSELL,
                self.OBPs_df.ORDERNO,
                self.OBPs_df.ACTION,
                self.OBPs_df.PRICE,
                self.OBPs_df.VOLUME,
                self.OBPs_df.TRADEPRICE,
                self.OBPs_df.OBPLOTNO,
                self.OBPs_df.DATETIMEMLLS)]

In [15]:
# obp_atts_2000ms = OBPlots2OBPlots_Attributes(OrderBookPlots2002_2000ms_df)

In [16]:
def obp_atts2order_atts_df(obp_atts):
    order_atts_dic = {}
    for SECCODE in obp_atts:
        for DATE in obp_atts[SECCODE]:
            for OBPLOTNO in obp_atts[SECCODE][DATE]:
                for NO in obp_atts[SECCODE][DATE][OBPLOTNO]:
                    order_atts_dic[NO] = {}
                    if NO != 'PREV':
#                         print(NO)
#                         print(obp_atts[SECCODE][DATE][OBPLOTNO][NO])
                        order_atts_dic[NO] = obp_atts[SECCODE][DATE][OBPLOTNO][NO]
                        order_atts_dic[NO]['NO'] = NO
                        order_atts_dic[NO]['OBPLOTNO'] = OBPLOTNO
                        order_atts_dic[NO]['DATE'] = DATE
                        order_atts_dic[NO]['SECCODE'] = SECCODE

    order_atts_df = pd.DataFrame.from_dict(order_atts_dic, "index")
    order_atts_df.sort_values(by=['DATETIMEMLLS', 'NO'], inplace=True)
    return(order_atts_df)

In [17]:
# obp_atts_2000ms_df = obp_atts2order_atts_df(obp_atts_2000ms.obp_atts_dic)

In [18]:
# obp_atts_2002 = OBPlots2OBPlots_Attributes(OrderBookPlots2002_df)
# obp_atts_2002_df = obp_atts2order_atts_df(obp_atts_2002.obp_atts_dic)
# obp_atts_2002_df

In [19]:
obp_atts_2007 = OBPlots2OBPlots_Attributes(OrderBookPlots2007_df)
obp_atts_2007_df = obp_atts2order_atts_df(obp_atts_2007.obp_atts_dic)
obp_atts_2007_df

,ATT,VAL,DATETIMEMLLS,NO,OBPLOTNO,DATE,SECCODE
13,SOVOL,14649812.50,2007-10-08 10:30:00.000,13,0,2007-10-08,LKOH
46,SOVOL,33210.00,2007-10-08 10:30:00.001,46,2,2007-10-08,LKOH
74,BOVOL,99500.00,2007-10-08 10:30:00.003,74,4,2007-10-08,LKOH
76,SOVOL,22000.00,2007-10-08 10:30:00.004,76,6,2007-10-08,LKOH
77,BOVOL,12660.00,2007-10-08 10:30:00.005,77,8,2007-10-08,LKOH
...,...,...,...,...,...,...,...
910328,STVOL,2176.02,2007-10-08 17:44:58.812,910328,79201,2007-10-08,LKOH
910329,STVOL,87067.32,2007-10-08 17:44:58.875,910329,79201,2007-10-08,LKOH
910330,BTVOL,87067.32,2007-10-08 17:44:58.938,910330,79201,2007-10-08,LKOH
910337,CSOVOL,21774.40,2007-10-08 17:44:59.000,910337,78407,2007-10-08,LKOH


In [20]:
def obp_atts2buy_sell_obp_df(order_atts_df):
    def buy_sell_obp(gr):
        buy_minus_sell_vol = gr[gr.ATT == 'BOVOL']['VAL'].sum() - gr[gr.ATT == 'SOVOL']['VAL'].sum()
        if buy_minus_sell_vol > 0:
            return('B')
        elif buy_minus_sell_vol < 0:
            return('S')
        else:
            return('N')
    buy_sell_obp_ser = order_atts_df.groupby(['DATE', 'SECCODE', 'OBPLOTNO']).apply(lambda gr: buy_sell_obp(gr))
    df = pd.DataFrame.from_dict({'BUYSELLOBP': buy_sell_obp_ser})
    df.reset_index(inplace=True)
    return(df)

In [21]:
obp_atts_df = obp_atts2buy_sell_obp_df(obp_atts_2007_df)
obp_atts_df

,DATE,SECCODE,OBPLOTNO,BUYSELLOBP
0,2007-10-08,LKOH,0,S
1,2007-10-08,LKOH,2,S
2,2007-10-08,LKOH,4,B
3,2007-10-08,LKOH,6,S
4,2007-10-08,LKOH,8,B
...,...,...,...,...
19716,2007-10-08,LKOH,79307,S
19717,2007-10-08,LKOH,79314,B
19718,2007-10-08,LKOH,79320,B
19719,2007-10-08,LKOH,79324,B


In [22]:
obp_atts_df[obp_atts_df.BUYSELLOBP == 'N']

,DATE,SECCODE,OBPLOTNO,BUYSELLOBP
1296,2007-10-08,LKOH,3744,N
1651,2007-10-08,LKOH,5278,N
1855,2007-10-08,LKOH,6103,N
2090,2007-10-08,LKOH,7096,N
2113,2007-10-08,LKOH,7200,N
...,...,...,...,...
19393,2007-10-08,LKOH,77874,N
19449,2007-10-08,LKOH,78058,N
19551,2007-10-08,LKOH,78536,N
19680,2007-10-08,LKOH,79041,N
